In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from os.path import realpath
sys.path.insert(0, realpath('../src/'))

## Load & Preprocess Dataset

In [3]:
from data.gerber_chai import ImpArgDataset

imp_arg_dataset = ImpArgDataset(
    corenlp_root='/home/pengxiang/workspace/corpora/wsj_corenlp/20170613',
    split_by_original=False, include_non_head_entity=True, suppress_warning=True)

imp_arg_dataset.read_dataset('../data/gerber_chai/implicit_argument_annotations.xml')

2018-05-01 15:56:18,780 - INFO - 'pattern' package not found; tag filters are not available for English
2018-05-01 15:56:18,829 - INFO - Reading implicit argument dataset from ../data/gerber_chai/implicit_argument_annotations.xml
2018-05-01 15:56:19,283 - INFO - Found 1247 instances


In [4]:
imp_arg_dataset.build_predicates()

2018-05-01 15:56:24,672 - INFO - Building PTBReader from /home/pengxiang/workspace/corpora/penn-treebank-rel3/parsed/mrg/wsj
2018-05-01 15:56:24,674 - INFO - Found 2312 files
2018-05-01 15:56:24,675 - INFO - Building NombankReader from /home/pengxiang/workspace/corpora/nombank.1.0/nombank.1.0_sorted_old
2018-05-01 15:56:26,365 - INFO - Found 114574 instances
2018-05-01 15:56:26,366 - INFO - Building index by fileid for NombankReader
2018-05-01 15:56:28,977 - INFO - Building predicate mapping
2018-05-01 15:56:31,259 - INFO - Building CoreNLP Reader from /home/pengxiang/workspace/corpora/wsj_corenlp/20170613
2018-05-01 15:56:50,943 - INFO - Done
2018-05-01 15:56:50,944 - INFO - Building predicates
2018-05-01 15:56:51,016 - INFO - Checking explicit arguments with Nombank instances
2018-05-01 15:56:51,034 - INFO - Parsing all implicit and explicit arguments
2018-05-01 15:57:05,572 - INFO - Done


In [5]:
imp_arg_dataset.add_candidates()

2018-05-01 15:57:05,592 - INFO - Building PropbankReader from /home/pengxiang/workspace/corpora/propbank-LDC2004T14/data/prop.txt
2018-05-01 15:57:07,638 - INFO - Found 112917 instances
2018-05-01 15:57:07,639 - INFO - Building index by fileid for PropbankReader
2018-05-01 15:57:11,205 - INFO - Building candidate dict from Propbank and Nombank
2018-05-01 15:58:57,982 - INFO - Done
2018-05-01 15:58:57,983 - INFO - Adding candidates to predicates


In [6]:
imp_arg_dataset.build_rich_predicates(use_corenlp_token=True, labeled_arg_only=False)

2018-05-01 15:58:58,120 - INFO - Building rich predicates with CoreNLP tokens
2018-05-01 15:59:06,868 - INFO - Done


In [7]:
imp_arg_dataset.print_stats()

+-----------------------------------------------------------------------+
|      Pred.            # Pred.         # Imp.Arg.       # Imp./pred.   |
+=======================================================================+
|      sale               184               182              0.99       |
|      price              216               139              0.64       |
|       bid               88                124              1.41       |
|    investor             160               108              0.68       |
|      cost               101               86               0.85       |
|      loan               84                82               0.98       |
|      plan               100               77               0.77       |
|      loss               104               62               0.60       |
|      fund               108               56               0.52       |
|   investment            102               54               0.53       |
|                                     

## Load event composition models and fill / no-fill classifier

In [8]:
from model.event_comp import EventCompositionModel
from os.path import join
from config import cfg

model_8M = EventCompositionModel.load_model(
    join(cfg.model_root, '20170609/fine_tuning_full/iter_19'))
model_8M_multi = EventCompositionModel.load_model(
    join(cfg.model_root, '20171127/fine_tuning_full/iter_17'))
model_8M_multi_salience = EventCompositionModel.load_model(
    join(cfg.model_root, '20170902/fine_tuning_full/iter_14'))

model_40M = EventCompositionModel.load_model(
    join(cfg.model_root, '20170611/fine_tuning_full/iter_19'))
model_40M_multi = EventCompositionModel.load_model(
    join(cfg.model_root, '20171203_1_tacc/fine_tuning_full/iter_14'))
model_40M_multi_salience = EventCompositionModel.load_model(
    join(cfg.model_root, '20171205/fine_tuning_full/iter_11'))

Using cuDNN version 7004 on context None
Mapped name None to device cuda: GeForce GTX 1080 Ti (0000:01:00.0)
2018-05-01 15:59:08,733 - INFO - loading word counts from /home/pengxiang/workspace/corpora/enwiki-20160901/event_comp_training/results/20170609/fine_tuning_full/iter_19/word2vec.vocab
2018-05-01 15:59:08,826 - INFO - loading projection weights from /home/pengxiang/workspace/corpora/enwiki-20160901/event_comp_training/results/20170609/fine_tuning_full/iter_19/word2vec.bin
2018-05-01 15:59:09,264 - INFO - loaded (53345, 300) matrix from /home/pengxiang/workspace/corpora/enwiki-20160901/event_comp_training/results/20170609/fine_tuning_full/iter_19/word2vec.bin
2018-05-01 15:59:09,267 - INFO - precomputing L2-norms of word weight vectors
2018-05-01 15:59:10,805 - INFO - loading word counts from /home/pengxiang/workspace/corpora/enwiki-20160901/event_comp_training/results/20171127/fine_tuning_full/iter_17/word2vec.vocab
2018-05-01 15:59:10,893 - INFO - loading projection weights fro

In [9]:
import pickle as pkl
missing_labels_mapping = pkl.load(open(
    '../data/gerber_chai/binary_classifier/missing_labels_14_features.pkl', 'r'))

## Evaluation

### EventComp-8M

In [10]:
imp_arg_dataset.compute_coherence_score(
    model_8M, use_max_score=True, missing_labels_mapping=None)
imp_arg_dataset.cross_val(comp_wo_arg=False)
imp_arg_dataset.print_eval_stats()

Processed: 100%|████████████████████████████████████████████████| 1247/1247 [00:38<00:00, 32.07it/s]
2018-05-01 15:59:55,683 - INFO - Predicates with no context events:
2018-05-01 15:59:55,684 - INFO - Predicate #47: sale, missing_imp_args = 4, imp_args = 1


+----------------------------------------------------------------+
|   Pred.      # Pred.   # Imp.Arg.   Precision   Recall    F1   |
+================================================================+
| sale         184       182          11.24       33.96    16.89 |
| price        216       139          10.81       36.26    16.66 |
| bid          88        124          12.28       18.61    14.79 |
| investor     160       108          4.93        20.20    7.93  |
| cost         101       86           6.03        16.12    8.77  |
| loan         84        82           8.93        25.14    13.17 |
| plan         100       77           12.55       33.40    18.24 |
| loss         104       62           9.65        33.46    14.98 |
| fund         108       56           3.87        18.71    6.41  |
| investment   102       54           9.08        41.18    14.87 |
|                                                                |
| Overall      1247      970          8.90        27.91    13.

### EventComp-8M + fill / no-fill classifier

In [11]:
imp_arg_dataset.compute_coherence_score(
    model_8M, use_max_score=True, missing_labels_mapping=missing_labels_mapping)
for rich_predicate in imp_arg_dataset.all_rich_predicates:
    rich_predicate.eval(0.0, comp_wo_arg=False)
imp_arg_dataset.print_eval_stats()

Processed: 100%|███████████████████████████████████████████████| 1247/1247 [00:10<00:00, 119.60it/s]
2018-05-01 16:00:09,649 - INFO - Predicates with no context events:
2018-05-01 16:00:09,651 - INFO - Predicate #47: sale, missing_imp_args = 4, imp_args = 1


+----------------------------------------------------------------+
|   Pred.      # Pred.   # Imp.Arg.   Precision   Recall    F1   |
+================================================================+
| sale         184       182          25.70       27.11    26.39 |
| price        216       139          24.86       28.61    26.60 |
| bid          88        124          17.98       20.44    19.13 |
| investor     160       108          14.14       15.58    14.83 |
| cost         101       86           13.90       15.35    14.59 |
| loan         84        82           20.99       21.25    21.12 |
| plan         100       77           33.81       34.69    34.25 |
| loss         104       62           35.55       26.95    30.66 |
| fund         108       56           9.05        5.65     6.96  |
| investment   102       54           23.28       13.80    17.33 |
|                                                                |
| Overall      1247      970          21.97       22.26    22.

### EventComp-8M + fill / no-fill classifier + multi-arg training

In [12]:
imp_arg_dataset.compute_coherence_score(
    model_8M_multi, use_max_score=True, missing_labels_mapping=missing_labels_mapping)
for rich_predicate in imp_arg_dataset.all_rich_predicates:
    rich_predicate.eval(0.0, comp_wo_arg=False)
imp_arg_dataset.print_eval_stats()

Processed: 100%|███████████████████████████████████████████████| 1247/1247 [00:10<00:00, 118.20it/s]
2018-05-01 16:00:20,965 - INFO - Predicates with no context events:
2018-05-01 16:00:20,966 - INFO - Predicate #47: sale, missing_imp_args = 4, imp_args = 1


+----------------------------------------------------------------+
|   Pred.      # Pred.   # Imp.Arg.   Precision   Recall    F1   |
+================================================================+
| sale         184       182          45.94       48.46    47.16 |
| price        216       139          31.40       36.14    33.60 |
| bid          88        124          45.42       51.64    48.33 |
| investor     160       108          41.43       45.65    43.44 |
| cost         101       86           35.26       38.95    37.01 |
| loan         84        82           49.75       50.36    50.05 |
| plan         100       77           62.39       64.02    63.19 |
| loss         104       62           57.48       43.57    49.57 |
| fund         108       56           41.90       26.19    32.23 |
| investment   102       54           31.24       18.51    23.25 |
|                                                                |
| Overall      1247      970          43.49       44.08    43.

### EventComp-8M + fill / no-fill classifier + multi-arg training + entity salience

In [13]:
imp_arg_dataset.compute_coherence_score(
    model_8M_multi_salience, use_max_score=True, missing_labels_mapping=missing_labels_mapping)
for rich_predicate in imp_arg_dataset.all_rich_predicates:
    rich_predicate.eval(0.0, comp_wo_arg=False)
imp_arg_dataset.print_eval_stats()

Processed: 100%|███████████████████████████████████████████████| 1247/1247 [00:11<00:00, 111.37it/s]
2018-05-01 16:00:32,940 - INFO - Predicates with no context events:
2018-05-01 16:00:32,942 - INFO - Predicate #47: sale, missing_imp_args = 4, imp_args = 1


+----------------------------------------------------------------+
|   Pred.      # Pred.   # Imp.Arg.   Precision   Recall    F1   |
+================================================================+
| sale         184       182          44.71       47.16    45.90 |
| price        216       139          39.77       45.77    42.56 |
| bid          88        124          46.44       52.80    49.41 |
| investor     160       108          43.68       48.13    45.80 |
| cost         101       86           40.18       44.39    42.18 |
| loan         84        82           49.33       49.93    49.63 |
| plan         100       77           64.58       66.26    65.41 |
| loss         104       62           58.61       44.43    50.54 |
| fund         108       56           39.05       24.40    30.04 |
| investment   102       54           35.55       21.06    26.45 |
|                                                                |
| Overall      1247      970          45.74       46.36    46.

### EventComp-40M

In [14]:
imp_arg_dataset.compute_coherence_score(
    model_40M, use_max_score=True, missing_labels_mapping=None)
imp_arg_dataset.cross_val(comp_wo_arg=False)
imp_arg_dataset.print_eval_stats()

Processed: 100%|████████████████████████████████████████████████| 1247/1247 [00:38<00:00, 32.36it/s]
2018-05-01 16:01:12,262 - INFO - Predicates with no context events:
2018-05-01 16:01:12,263 - INFO - Predicate #47: sale, missing_imp_args = 4, imp_args = 1


+----------------------------------------------------------------+
|   Pred.      # Pred.   # Imp.Arg.   Precision   Recall    F1   |
+================================================================+
| sale         184       182          9.38        30.92    14.39 |
| price        216       139          10.17       34.83    15.75 |
| bid          88        124          16.84       24.99    20.12 |
| investor     160       108          7.23        30.05    11.65 |
| cost         101       86           6.87        19.26    10.13 |
| loan         84        82           10.18       29.93    15.19 |
| plan         100       77           13.74       35.50    19.81 |
| loss         104       62           11.27       41.79    17.75 |
| fund         108       56           3.47        15.94    5.70  |
| investment   102       54           8.85        41.15    14.57 |
|                                                                |
| Overall      1247      970          9.39        30.27    14.

### EventComp-40M + fill / no-fill classifier

In [15]:
imp_arg_dataset.compute_coherence_score(
    model_40M, use_max_score=True, missing_labels_mapping=missing_labels_mapping)
for rich_predicate in imp_arg_dataset.all_rich_predicates:
    rich_predicate.eval(0.0, comp_wo_arg=False)
imp_arg_dataset.print_eval_stats()

Processed: 100%|███████████████████████████████████████████████| 1247/1247 [00:10<00:00, 119.76it/s]
2018-05-01 16:01:26,312 - INFO - Predicates with no context events:
2018-05-01 16:01:26,313 - INFO - Predicate #47: sale, missing_imp_args = 4, imp_args = 1


+----------------------------------------------------------------+
|   Pred.      # Pred.   # Imp.Arg.   Precision   Recall    F1   |
+================================================================+
| sale         184       182          30.21       31.87    31.02 |
| price        216       139          24.23       27.90    25.94 |
| bid          88        124          21.13       24.02    22.48 |
| investor     160       108          17.82       19.64    18.68 |
| cost         101       86           14.05       15.52    14.75 |
| loan         84        82           25.95       26.27    26.11 |
| plan         100       77           33.10       33.96    33.52 |
| loss         104       62           35.99       27.28    31.04 |
| fund         108       56           6.83        4.27     5.25  |
| investment   102       54           15.58       9.23     11.59 |
|                                                                |
| Overall      1247      970          23.71       24.03    23.

### EventComp-40M + fill / no-fill classifier + multi-arg training

In [16]:
imp_arg_dataset.compute_coherence_score(
    model_40M_multi, use_max_score=True, missing_labels_mapping=missing_labels_mapping)
for rich_predicate in imp_arg_dataset.all_rich_predicates:
    rich_predicate.eval(0.0, comp_wo_arg=False)
imp_arg_dataset.print_eval_stats()

Processed: 100%|███████████████████████████████████████████████| 1247/1247 [00:10<00:00, 116.55it/s]
2018-05-01 16:01:37,838 - INFO - Predicates with no context events:
2018-05-01 16:01:37,840 - INFO - Predicate #47: sale, missing_imp_args = 4, imp_args = 1


+----------------------------------------------------------------+
|   Pred.      # Pred.   # Imp.Arg.   Precision   Recall    F1   |
+================================================================+
| sale         184       182          45.98       48.51    47.21 |
| price        216       139          31.03       35.72    33.21 |
| bid          88        124          44.57       50.69    47.43 |
| investor     160       108          55.20       60.83    57.88 |
| cost         101       86           49.34       54.50    51.79 |
| loan         84        82           52.98       53.62    53.30 |
| plan         100       77           65.28       66.98    66.12 |
| loss         104       62           60.54       45.89    52.21 |
| fund         108       56           29.37       18.35    22.59 |
| investment   102       54           34.50       20.45    25.68 |
|                                                                |
| Overall      1247      970          46.66       47.28    46.

### EventComp-40M + fill / no-fill classifier + multi-arg training + entity salience

In [17]:
imp_arg_dataset.compute_coherence_score(
    model_40M_multi_salience, use_max_score=True, missing_labels_mapping=missing_labels_mapping)
for rich_predicate in imp_arg_dataset.all_rich_predicates:
    rich_predicate.eval(0.0, comp_wo_arg=False)
imp_arg_dataset.print_eval_stats()

Processed: 100%|███████████████████████████████████████████████| 1247/1247 [00:11<00:00, 104.23it/s]
2018-05-01 16:01:50,687 - INFO - Predicates with no context events:
2018-05-01 16:01:50,688 - INFO - Predicate #47: sale, missing_imp_args = 4, imp_args = 1


+----------------------------------------------------------------+
|   Pred.      # Pred.   # Imp.Arg.   Precision   Recall    F1   |
+================================================================+
| sale         184       182          46.02       48.55    47.25 |
| price        216       139          40.72       46.87    43.58 |
| bid          88        124          49.23       55.98    52.39 |
| investor     160       108          50.74       55.91    53.20 |
| cost         101       86           53.92       59.56    56.60 |
| loan         84        82           51.29       51.91    51.60 |
| plan         100       77           64.51       66.19    65.34 |
| loss         104       62           63.88       48.42    55.09 |
| fund         108       56           42.34       26.47    32.57 |
| investment   102       54           35.55       21.06    26.45 |
|                                                                |
| Overall      1247      970          49.26       49.93    49.